In [1]:
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
import squarify
import plotly.graph_objs as go
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
%matplotlib inline
from IPython.display import display

In [2]:
data_export = pd.read_csv("Dataset/india-trade-data/2018-2010_export.csv")
data_import = pd.read_csv("Dataset/india-trade-data/2018-2010_import.csv")
print(data_export.shape, data_import.shape)

(137023, 5) (76124, 5)


In [3]:
def cleanupImpute(data_df):
    columns = ["value", "country"]
    print(data_df.shape)
    data_df["country"] = data_df['country'].apply(lambda x : np.NaN if x == "UNSPECIFIED" else x)
    data_df["value"] = data_df["value"].fillna(data_df.groupby("Commodity")["value"].transform('mean'))
    data_df = data_df.replace(0, np.nan).dropna(axis=0, how='any', subset=columns)
    print(data_df.shape)
    data_df.year = pd.Categorical(data_df.year)
    data_df.drop_duplicates(keep="first",inplace=True)
    return data_df

data_import = cleanupImpute(data_import)
data_export = cleanupImpute(data_export)

display(data_import.info())
display(data_export.info())

(76124, 5)
(66779, 5)
(137023, 5)
(120510, 5)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 65882 entries, 1 to 76123
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   HSCode     65882 non-null  int64   
 1   Commodity  65882 non-null  object  
 2   value      65882 non-null  float64 
 3   country    65882 non-null  object  
 4   year       65882 non-null  category
dtypes: category(1), float64(1), int64(1), object(2)
memory usage: 2.6+ MB


None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120510 entries, 0 to 137022
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype   
---  ------     --------------   -----   
 0   HSCode     120510 non-null  int64   
 1   Commodity  120510 non-null  object  
 2   value      120510 non-null  float64 
 3   country    120510 non-null  object  
 4   year       120510 non-null  category
dtypes: category(1), float64(1), int64(1), object(2)
memory usage: 4.7+ MB


None

In [4]:
yearly_imports = data_import.groupby('year').agg({'value':'sum'})
yearly_exports = data_export.groupby('year').agg({'value':'sum'})
yearly_imports['deficit'] = yearly_exports.value - yearly_imports.value
yearly_exports['surplus'] = yearly_imports.value - yearly_exports.value

countrywise_imports = data_import.groupby('country').agg({'value':'sum'})
countrywise_imports = countrywise_imports.sort_values(by='value', ascending = True)
countrywise_imports = countrywise_imports[:10]

countrywise_exports = data_export.groupby('country').agg({'value':'sum'})
countrywise_exports = countrywise_exports.sort_values(by='value', ascending = True)
countrywise_exports = countrywise_exports[:10]

<div class="alert alert-success">

<b>Imports & Exports</b>:
Interactive plots for country wise import & exports
    
</div>

In [5]:
fig_cw_imp = px.bar(countrywise_imports, x=countrywise_imports.index, y="value", text="value",hover_data=["value"])
fig_cw_imp.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig_cw_imp.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig_cw_imp.show()

fig_cw_exp = px.bar(countrywise_exports, x=countrywise_exports.index, y="value", text="value", hover_data=["value"])
fig_cw_exp.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig_cw_exp.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig_cw_exp.show()

<div class="alert alert-success">

<b>Deficits & Surplus</b>:
Interactive plots for country wise deficits and surplus
<ul>
    <li>Interactive country wise deficit and surplus plots, for top 10 countries</li>
</ul>
    
</div>

In [6]:
cw_imports = data_import.groupby(["country"]).agg({'value':'sum'})
cw_exports = data_export.groupby(["country"]).agg({'value':'sum'})
cw_imports['deficit'] = cw_exports.value - cw_imports.value
cw_exports['surplus'] = cw_exports.value - cw_imports.value
cw_imports = cw_imports.sort_values(by='deficit', ascending = True)
cw_exports = cw_exports.sort_values(by='surplus', ascending = False)
cw_exports = cw_exports[:10]
cw_imports = cw_imports[:10]

fig_cw_def = px.bar(cw_imports, x=cw_imports.index, y="deficit", text="deficit",hover_data=["deficit"])
fig_cw_def.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig_cw_def.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig_cw_def.show()

fig_cw_sur = px.bar(cw_exports, x=cw_exports.index, y="surplus", text="surplus", hover_data=["surplus"])
fig_cw_sur.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig_cw_sur.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig_cw_sur.show()

<div class="alert alert-success">

<b>Trends For Deficit & Surplus Countries</b>:
<ul>
    <li>Create interactive visualisations to analyse trends for top 10 deficit and surplus countries.</li>
</ul>
    
</div>

In [7]:
cw_imp = data_import.groupby(["country", "year"]).agg({'value':'sum'})
cw_exp = data_export.groupby(["country", "year"]).agg({'value':'sum'})
cw_imp['value_exp'] = cw_exp.value
cw_imp['deficit'] = cw_exp.value - cw_imp.value
cw_exp['value_imp'] = cw_imp.value
cw_exp['surplus'] = cw_exp.value - cw_imp.value

cw_imports = data_import.groupby(["country"]).agg({'value':'sum'})
cw_exports = data_export.groupby(["country"]).agg({'value':'sum'})
cw_imports['deficit'] = cw_exports.value - cw_imports.value
cw_exports['surplus'] = cw_exports.value - cw_imports.value

top10def = list(cw_imports.sort_values(by='deficit', ascending = True)[:10].index)
top10sur = list(cw_exports.sort_values(by='surplus', ascending = False)[:10].index)

not_top10def = list(cw_imports.sort_values(by='deficit', ascending = True)[10:].index)
not_top10sur = list(cw_exports.sort_values(by='surplus', ascending = False)[10:].index)

cw_imp = cw_imp.reset_index()
cw_imp = cw_imp.set_index("country")

cw_exp = cw_exp.reset_index()
cw_exp = cw_exp.set_index("country")


cw_imp["Total_Deficit"] = cw_imports.deficit
cw_exp["Total_Surplus"] = cw_exports.surplus

cw_imp = cw_imp.sort_values(by='Total_Deficit', ascending = True)
cw_exp = cw_exp.sort_values(by='Total_Surplus', ascending = False)

cw_imp = cw_imp.drop(labels=not_top10def)
cw_exp = cw_exp.drop(labels=not_top10sur)

cw_imp = cw_imp.groupby("country")
cw_exp = cw_exp.groupby("country")

fig_def = go.Figure()
for d in top10def:
    cur_df = cw_imp.get_group(d)
    cur_df = cur_df.sort_values(by='year', ascending = True)
    fig_def.add_trace(go.Scatter(dict(y=cur_df.deficit, x=cur_df.year, name=d, mode='lines+markers')))
    
fig_sur = go.Figure()
for s in top10sur:
    cur_df = cw_exp.get_group(s)
    cur_df = cur_df.sort_values(by='year', ascending = True)
    fig_sur.add_trace(go.Scatter(dict(y=cur_df.surplus, x=cur_df.year, name=s, mode='lines+markers')))

    
fig_def.update_layout(
    title=go.layout.Title(
        text="Yearwise Deficit of Top 10 Countries",
        xref="paper",
        x=0
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text="Year",
            font=dict(
                family="Courier New, monospace",
                size=18,
                color="#7f7f7f"
            )
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text="Value",
            font=dict(
                family="Courier New, monospace",
                size=18,
                color="#7f7f7f"
            )
        )
    )
)

fig_sur.update_layout(
    title=go.layout.Title(
        text="Yearwise Surplus of Top 10 Countries",
        xref="paper",
        x=0
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text="Year",
            font=dict(
                family="Courier New, monospace",
                size=18,
                color="#7f7f7f"
            )
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text="Value",
            font=dict(
                family="Courier New, monospace",
                size=18,
                color="#7f7f7f"
            )
        )
    )
)

fig_def.show()
fig_sur.show()

<div class="alert alert-success">

<b>Trends For Trade-Balanced Countries</b>:
<ul>
    <li>Interactive visualisations to analyse trends for the most trade-balanced 10 nations?</li>
</ul>
    
</div>

In [8]:
cw_i = data_import.groupby(["country", "year"]).agg({'value':'sum'})
cw_e = data_export.groupby(["country", "year"]).agg({'value':'sum'})

cw_i['deficit'] = cw_e.value - cw_i.value

cw_imports = data_import.groupby(["country"]).agg({'value':'sum'})
cw_imports['deficit'] = abs(cw_exports.value - cw_imports.value)

top10bal = list(cw_imports.sort_values(by='deficit', ascending = True)[:10].index)
not_top10bal = list(cw_imports.sort_values(by='deficit', ascending = True)[10:].index)

cw_i = cw_i.reset_index()
cw_i = cw_i.set_index("country")
cw_i["Total_Deficit"] = cw_imports.deficit
cw_i["Total_Deficit"] = abs(round(cw_i["Total_Deficit"], 2))
cw_i = cw_i.sort_values(by='Total_Deficit', ascending = True)
cw_i = cw_i.drop(labels=not_top10bal)

cw_i = cw_i.groupby("country")

fig_bal = go.Figure()
for b in top10bal:
    cur_df = cw_i.get_group(b)
    cur_df = cur_df.sort_values(by='year', ascending = True)
    fig_bal.add_trace(go.Scatter(dict(y=cur_df.deficit, x=cur_df.year, name=b, mode='lines+markers')))
    
fig_bal.update_layout(
    title=go.layout.Title(
        text="Top 10 Trade-Balanced Countries",
        xref="paper",
        x=0
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text="Year",
            font=dict(
                family="Courier New, monospace",
                size=18,
                color="#7f7f7f"
            )
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text="Deficit",
            font=dict(
                family="Courier New, monospace",
                size=18,
                color="#7f7f7f"
            )
        )
    )
)

fig_bal.show()

<div class="alert alert-success">

<b> Trade Balanced Nations Every Year </b>:
<ul>
    <li>Identifying the top 3 trade-balanced nations, every year</li>
</ul>
    
</div>

In [9]:
cw_i = data_import.groupby(["country", "year"]).agg({'value':'sum'})
cw_e = data_export.groupby(["country", "year"]).agg({'value':'sum'})
cw_i['deficit'] = abs(round((cw_e.value - cw_i.value),2))
cw_i = cw_i.reset_index()
cw_i = cw_i.groupby("year")

for i in range(2010,2019):
    cur = cw_i.get_group(i)
    print("\nYear : ", i)
    cur = cur.sort_values(by="deficit")[:3]
    top3 = list(cur["country"])
    print(top3)


Year :  2010
['NAMIBIA', 'FALKLAND IS', 'DOMINIC REP']

Year :  2011
['NETHERLANDANTIL', 'UZBEKISTAN', 'UGANDA']

Year :  2012
['MARSHALL ISLAND', 'GUINEA BISSAU', 'VIRGIN IS US']

Year :  2013
['RWANDA', 'ESTONIA', 'NIUE IS']

Year :  2014
['EQUTL GUINEA', 'FAROE IS.', 'BAHARAIN IS']

Year :  2015
['FR S ANT TR', 'TUNISIA', 'URUGUAY']

Year :  2016
['BOSNIA-HRZGOVIN', 'SOUTH SUDAN ', 'SIERRA LEONE']

Year :  2017
['NORFOLK IS', 'ST HELENA', 'ECUADOR']

Year :  2018
['ROMANIA', 'TUVALU', 'NORWAY']


<div class="alert alert-success">

<b>Expensive Imports</b>:
<ul>
    <li>Import Value Vs HSCode(Commodity Code)</li>
</ul>
    
</div>

In [10]:
expensive_import = data_import[data_import.value>1000]
fig = px.box(expensive_import, x="HSCode", y="value")
fig.show()

In [11]:
hscode = data_import
hscode = hscode[["HSCode", "Commodity"]]
hscode = hscode.drop_duplicates(subset="HSCode", keep='first')
hscode = hscode.set_index("HSCode")
df = expensive_import.groupby(['HSCode']).agg({'value': 'sum'})
df = df.sort_values(by='value')
df["commodity"] = hscode.Commodity
df = df.reset_index()
fig = px.treemap(df, path=['HSCode'], values=df.value, hover_data=['commodity'])
fig.update_layout(title="Expensive Imports HSCode Share")
fig.show()

<div class="alert alert-success">

<b>Country Analysis</b>:
<ul>
    <li>Expensive Imports Countrywise Share</li>
</ul>
    
</div>

In [12]:
total_deficit = data_import.groupby(["country"]).agg({'value':'sum'})
cw = data_export.groupby(["country"]).agg({'value':'sum'})
total_deficit["Total_Deficit"] = cw.value - total_deficit.value


deficit_recent = data_import.groupby(["country", "year"]).agg({'value':'sum'})
cyw = data_export.groupby(["country", "year"]).agg({'value':'sum'})
deficit_recent["Deficit_Year"] = cyw.value - deficit_recent.value
deficit_recent = deficit_recent.reset_index()
deficit_recent["year"] = list(map(int, list(deficit_recent["year"])))
deficit_recent = deficit_recent.groupby(["country"]).apply(lambda x: x.nlargest(1,['year'])).reset_index(drop=True)
deficit_recent = deficit_recent.set_index("country")

df1 = expensive_import.groupby(['country']).agg({'value': 'sum'})
df1 = df1.sort_values(by='value')
df1["Total_Deficit"] = total_deficit.Total_Deficit
df1["Recent_Deficit"] = deficit_recent.Deficit_Year 
df1 = df1.reset_index()

fig1 = px.treemap(df1, path=['country'], values=df1.value, hover_data=["Total_Deficit", "Recent_Deficit"])
fig1.update_layout(title="Expensive Imports Countrywise Share")
fig1.show()